In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import string
import math

In [2]:
with open('WSJ_training.pos') as f:    # Training data from Wall Street Journal
    training_corpus = f.readlines()    

    print("Training corpus list:", len(training_corpus))
print(training_corpus[:5])

Training corpus list: 989860
['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']


In [3]:
# Preparing vocabulary
# 1) Taking the words out from training set
words = [line.split('\t')[0] for line in training_corpus]
print('words',words[:7])

# 2) Creating default dictnary of word_count
word_count = defaultdict(int)
for i in words:
    word_count[i] += 1

# 3) vocab = words whos freqency is more than 2
vocab_l = [k for k, v in word_count.items() if (v > 1 and k != '\n')]
print('vocab',vocab_l[:7])

# 4) sort and give unique number 
vocab = {}
for i, word in enumerate(vocab_l): 
    vocab[word] = i 

words ['In', 'an', 'Oct.', '19', 'review', 'of', '``']
vocab ['In', 'an', 'Oct.', '19', 'review', 'of', '``']


In [4]:
# Assign tags to unknown words
def assign_unk(word):
    punct = set(string.punctuation)
    
    # Suffixes
    noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
    verb_suffix = ["ate", "ify", "ise", "ize"]
    adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
    adv_suffix = ["ward", "wards", "wise"]

    # digit
    if any(char.isdigit() for char in word):
        return "--unk_digit--"
    # punctuation character
    elif any(char in punct for char in word):
        return "--unk_punct--"
    # upper case character
    elif any(char.isupper() for char in word):
        return "--unk_upper--"
    # noun suffix
    elif any(word.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"
    # verb suffix
    elif any(word.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"
    # adjective suffix
    elif any(word.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"
    # adverb suffix
    elif any(word.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"  
    # If none of the previous criteria is met, return plain unknown
    return "--unk--"


In [5]:
def get_word_tag(line, vocab):
    if not line.split():             # If line is empty return placeholders for word and tag
        word = "--n--"
        tag = "--s--"
    else:
        word, tag = line.split()
        if word not in vocab: 
            word = assign_unk(word)
    return word, tag

In [6]:
def create_dictionaries(training_corpus, vocab):
    transition_counts = defaultdict(int)
    emission_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    for word_tag in training_corpus:
        word, tag = get_word_tag(word_tag,vocab)
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag,word)]+= 1
        tag_counts[tag] += 1
        prev_tag = tag
    return transition_counts, emission_counts, tag_counts

In [7]:
transition_counts, emission_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [8]:
print(len(transition_counts))
print("transition_count examples: ")
for ex in list(transition_counts.items())[:7]:
    print(ex)
print()
print(len(emission_counts))
print("emission_count examples: ")
for ex in list(emission_counts.items())[:7]:
    print (ex)
print()  
print(len(tag_counts))
print("tag_count examples: ")
for ex in list(tag_counts.items())[:7]:
    print (ex)

1421
transition_count examples: 
(('--s--', 'IN'), 5050)
(('IN', 'DT'), 32364)
(('DT', 'NNP'), 9044)
(('NNP', 'CD'), 1752)
(('CD', 'NN'), 7377)
(('NN', 'IN'), 32885)
(('IN', '``'), 546)

31140
emission_count examples: 
(('IN', 'In'), 1735)
(('DT', 'an'), 3142)
(('NNP', 'Oct.'), 317)
(('CD', '19'), 100)
(('NN', 'review'), 36)
(('IN', 'of'), 22925)
(('``', '``'), 6967)

46
tag_count examples: 
('IN', 98554)
('DT', 81842)
('NNP', 91466)
('CD', 36568)
('NN', 132935)
('``', 7092)
("''", 6919)


In [9]:
def create_transition_matrix(alpha, transition_counts, tag_counts):
    all_tags = list(tag_counts.keys())
    print(all_tags)
    num_tags = len(all_tags)
    
    A = np.zeros((num_tags,num_tags))
    
    trans_keys = set(list(transition_counts.keys()))

    for pre_tag in range(num_tags): 
        for tag in range(num_tags):
            count = 0
            key = (all_tags[pre_tag],all_tags[tag])

            if key in trans_keys: 
                count = transition_counts[key]
                
            prev_tag_count = tag_counts[all_tags[pre_tag]]
            
            A[pre_tag, tag] = (count + alpha) / (prev_tag_count + alpha * num_tags)
            
    transition_matrix = pd.DataFrame(A, index=all_tags, columns = all_tags)
    return transition_matrix

In [10]:
alpha = 0.01
transition_matrix = create_transition_matrix(alpha, transition_counts, tag_counts)
transition_matrix.iloc[:5, :]

['IN', 'DT', 'NNP', 'CD', 'NN', '``', "''", 'POS', '(', 'VBN', 'NNS', 'VBP', ',', 'CC', ')', 'VBD', 'RB', 'TO', '.', '--s--', 'VBZ', 'NNPS', 'PRP', 'PRP$', 'VB', 'JJ', 'MD', 'VBG', 'RBR', ':', 'WP', 'WDT', 'JJR', 'PDT', 'RBS', 'WRB', 'JJS', '$', 'RP', 'FW', 'EX', 'SYM', '#', 'LS', 'UH', 'WP$']


,IN,DT,NNP,CD,NN,``,'',POS,(,VBN,...,JJS,$,RP,FW,EX,SYM,#,LS,UH,WP$
IN,0.020415,0.328387,0.149694,0.059328,0.108915,0.005540,0.000102,0.000020,0.000345,0.004323,...,4.698012e-03,0.028005,1.024814e-05,0.000203,1.582983e-03,2.039482e-05,5.987552e-04,1.024814e-05,1.024814e-05,2.039482e-05
DT,0.009665,0.001576,0.110505,0.022922,0.474974,0.005547,0.000037,0.000037,0.000526,0.008419,...,9.310693e-03,0.009250,7.343377e-05,0.000257,1.221860e-07,1.221860e-07,1.589639e-04,1.221860e-07,2.455938e-05,1.221860e-07
NNP,0.040550,0.002405,0.376805,0.019155,0.058328,0.001115,0.002373,0.055693,0.003422,0.000842,...,1.093297e-07,0.000263,4.384121e-05,0.000492,2.197527e-05,1.104230e-05,1.093297e-07,1.093297e-07,1.093297e-07,1.093297e-07
CD,0.089996,0.028850,0.013099,0.201540,0.201731,0.000739,0.000383,0.000684,0.001668,0.003172,...,8.479985e-04,0.000110,2.734597e-07,0.000055,2.734597e-07,5.496540e-05,2.734597e-07,2.734597e-07,2.734597e-07,2.734597e-07
NN,0.247376,0.006778,0.009749,0.005980,0.122172,0.002385,0.005160,0.021838,0.001595,0.010727,...,4.520991e-05,0.000256,5.567363e-04,0.000053,1.204344e-04,3.016501e-05,1.512012e-05,7.522447e-08,7.522447e-08,1.806140e-04


In [11]:
def create_emission_matrix(alpha, emission_counts, tag_counts, vocab):
    all_tags = list(tag_counts.keys())
    
    num_tags = len(all_tags)
    num_words = len(vocab)
    
    B = np.zeros((num_tags,num_words))
    
    emis_keys = set(list(emission_counts.keys()))

    for tag in range(num_tags): 
        for word in range(num_words):
            count = 0
            key = (all_tags[tag],vocab[word])

            if key in emis_keys: 
                count = emission_counts[key]
                
            tag_count = tag_counts[all_tags[tag]]
            
            B[tag,word] = (count + alpha) / (tag_count + alpha * num_words)
            
    emission_matrix = pd.DataFrame(B, index=all_tags, columns = vocab)
    return emission_matrix

In [12]:
alpha = 0.01
emission_matrix = create_emission_matrix(alpha, emission_counts, tag_counts, list(vocab))
emission_matrix.iloc[:5,:]

,In,an,Oct.,19,review,of,``,The,Misanthrope,'',...,22.125,Post-Newsweek,Friend,Derr,Feinman,deadbeats,1926,Ibbotson,Blankenship,76.50
IN,1.756231e-02,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,2.320541e-01,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,...,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07,1.012231e-07
DT,1.218328e-07,3.828000e-02,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,8.278554e-02,1.218328e-07,1.218328e-07,...,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07,1.218328e-07
NNP,3.282311e-05,1.090469e-07,3.456895e-03,1.090469e-07,1.090469e-07,1.090469e-07,1.090469e-07,4.035825e-04,1.090469e-07,1.090469e-07,...,1.090469e-07,2.191842e-05,8.734656e-05,3.282311e-05,3.282311e-05,1.090469e-07,1.090469e-07,2.191842e-05,2.191842e-05,1.090469e-07
CD,2.716973e-07,2.716973e-07,2.716973e-07,2.717244e-03,2.716973e-07,2.716973e-07,2.716973e-07,2.716973e-07,2.716973e-07,2.716973e-07,...,5.461115e-05,2.716973e-07,2.716973e-07,2.716973e-07,2.716973e-07,2.716973e-07,5.461115e-05,2.716973e-07,2.716973e-07,5.461115e-05
NN,7.509048e-08,7.509048e-08,7.584139e-06,7.509048e-08,2.704008e-04,7.509048e-08,7.509048e-08,7.509048e-08,2.260224e-05,7.509048e-08,...,7.509048e-08,7.509048e-08,7.509048e-08,7.509048e-08,7.509048e-08,7.509048e-08,7.509048e-08,7.509048e-08,7.509048e-08,7.509048e-08


In [13]:
# Test Corpus

In [14]:
with open('WSJ_testing.pos') as f:    # Training data from Wall Street Journal
    testing_corpus = f.readlines()    

    print("Training corpus list:", len(testing_corpus))
print(testing_corpus[:5])
test_words = [line.split('\t')[0] for line in testing_corpus]

Training corpus list: 34199
['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n']


In [15]:
def preprocess(vocab, test_corpus):
    prep = []
    # Read data
    for word in test_corpus:
        # End of sentence
        if not word.split():
            word = "--n--"
            prep.append(word)
        # Handle unknown words
        elif word.strip() not in vocab:
            word = assign_unk(word)
            prep.append(word)
        else:
            prep.append(word.strip())

    return prep

In [16]:
corpus = preprocess(vocab, test_words) 

In [17]:
print(test_words[:13])
print(corpus[:13])

['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', 'vantage', 'points', 'this', 'week']
['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk_noun--', 'points', 'this', 'week']


In [18]:
def initialize(tag_counts, A, B, corpus, vocab):
    num_tags = len(tag_counts)
    all_tags = list(tag_counts.keys())
    
    best_probs = np.zeros((num_tags, len(corpus)))
    best_probs = pd.DataFrame(best_probs, index=all_tags, columns = corpus)
    best_paths = np.zeros((num_tags, len(corpus)))
    best_paths = pd.DataFrame(best_paths, index=all_tags, columns = corpus)
    
    s_idx = "--s--"
    
    for tag in all_tags:
        if A.loc[s_idx,tag] == 0: 
            best_probs.loc[tag,corpus[0]] = float("-inf")
        else:
            best_probs.loc[tag,corpus[0]] = math.log(A.loc[s_idx,tag]) + math.log(B.loc[tag,corpus[0]])
                       
    return best_probs, best_paths

In [19]:
best_probs, best_paths = initialize(tag_counts, transition_matrix, emission_matrix, corpus, vocab)

In [20]:
best_probs.iloc[:7,:]

,The,economy,'s,temperature,will,be,taken,from,several,--unk_noun--,...,we,may,not,see,them,here,with,us,.,--n--
IN,-18.171231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DT,-4.018855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NNP,-9.415879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CD,-19.624217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NN,-19.620495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
``,-16.089923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'',-21.300194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
def viterbi_forward(A, B,tag_counts, corpus, best_probs, best_paths, vocab):
    num_tags = len(tag_counts)
    all_tags = list(tag_counts.keys())
    
    for word in range(1,len(corpus[:200])): 
        for prev_tag in range(num_tags): 
            best_prob_word =  float("-inf")
            best_path_word = None
            for tag in range(num_tags):
                prob = best_probs.iloc[prev_tag,word-1] + math.log(A.iloc[prev_tag,tag]) + math.log(B.iloc[tag,word]) 
                if prob > best_prob_word: 
                    best_prob_word = prob
                    best_path_word = tag
            best_probs.iloc[prev_tag,word] = best_prob_word
            best_paths.iloc[prev_tag,word] = best_path_word
            
        ### END CODE HERE ###
    return best_probs, best_paths

In [24]:
best_probs, best_paths = viterbi_forward(transition_matrix, emission_matrix, tag_counts, corpus, best_probs, best_paths, vocab)

In [50]:
best_probs.iloc[:,:10]

,The,economy,'s,temperature,will,be,taken,from,several,--unk_noun--
IN,-18.171231,-22.547621,-30.114167,-38.846983,-49.279780,-54.632040,-59.878512,-63.483576,-76.398230,-85.646044
DT,-4.018855,-13.734344,-21.604422,-31.288208,-40.248309,-46.348335,-51.593511,-60.537675,-71.979632,-82.243278
NNP,-9.415879,-18.708761,-25.352173,-35.215519,-46.272806,-50.938798,-57.788199,-66.309756,-79.848899,-85.945717
CD,-19.624217,-26.432681,-36.435285,-43.945189,-53.761612,-57.630389,-64.891867,-70.929005,-83.227284,-91.147489
NN,-19.620495,-27.877425,-38.175383,-49.202785,-59.520714,-62.378346,-68.467776,-75.953380,-88.753166,-94.072927
``,-16.089923,-21.297870,-29.568625,-40.249038,-50.936152,-55.143405,-62.958997,-67.395618,-80.564589,-94.089581
'',-21.300194,-27.871686,-36.326700,-48.303717,-59.872436,-63.793481,-69.741353,-75.541519,-89.592095,-94.615813
POS,-28.900920,-40.536859,-48.467543,-58.017537,-67.110985,-74.464546,-78.961465,-89.039895,-100.615200,-109.041346
(,-17.607775,-23.505512,-30.294778,-39.661461,-50.553644,-54.506931,-58.039747,-63.166158,-76.540197,-88.418380
VBN,-19.675971,-25.620206,-34.926979,-45.293046,-56.240161,-58.705787,-64.076172,-69.249081,-82.678053,-88.971979


In [67]:
def viterbi_backward(best_probs, best_paths, corpus, tag_counts):
    all_tags = list(tag_counts.keys())
    num_tags = best_probs.shape[0]
    m = best_paths.shape[1]
    
    z = [None] * m
    best_prob_for_last_word = float('-inf')
    pred = [None] * m
    
    for k in range(num_tags):
        if best_probs.iloc[k,m-1] > best_prob_for_last_word: 
            best_prob_for_last_word = best_probs.iloc[k,m-1]
            z[m - 1] = k
            
    pred[m - 1] = all_tags[k]
    
    for i in range(m-1,-1,-1): 
        pos_tag_for_word_i = int(z[i])
        z[i - 1] = best_paths.iloc[pos_tag_for_word_i,i]
        pred[i - 1] = all_tags[int(z[i - 1])]
        
     ### END CODE HERE ###
    return pred

In [72]:
pred = viterbi_backward(best_probs.iloc[:,:100], best_paths.iloc[:,:100], corpus, tag_counts)

In [76]:
for i in range(100):
    print('Predicted POS for "{}" is "{}"'.format(corpus[i], pred[i]))

Predicted POS for "The" is "DT"
Predicted POS for "economy" is "NNP"
Predicted POS for "'s" is "CD"
Predicted POS for "temperature" is "NN"
Predicted POS for "will" is "IN"
Predicted POS for "be" is "``"
Predicted POS for "taken" is "DT"
Predicted POS for "from" is "NN"
Predicted POS for "several" is "''"
Predicted POS for "--unk_noun--" is "IN"
Predicted POS for "points" is "NNP"
Predicted POS for "this" is "POS"
Predicted POS for "week" is "NNP"
Predicted POS for "," is "NNP"
Predicted POS for "with" is "("
Predicted POS for "readings" is "VBP"
Predicted POS for "on" is "DT"
Predicted POS for "trade" is "NNP"
Predicted POS for "," is "IN"
Predicted POS for "output" is "NNP"
Predicted POS for "," is ","
Predicted POS for "housing" is "NN"
Predicted POS for "and" is "CC"
Predicted POS for "inflation" is "NNP"
Predicted POS for "." is ")"
Predicted POS for "--n--" is "NN"
Predicted POS for "The" is "NNP"
Predicted POS for "most" is "VBN"
Predicted POS for "troublesome" is "IN"
Predicted